In [1]:
import os
os.chdir('..')

In [2]:
import torch
from tqdm import tqdm
from src.pharmaconet.modeling import ModelingModule

/home/tsa87/anaconda3/envs/tacogfn/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
MODEL_PATH = 'model_weights/model.tar'
split_file = torch.load('dataset/pocket_to_ligands.pt')

In [4]:
score_threshold =  {
    'Hydrophobic': 0.12035781145095825,
    'PiStacking_P': 0.0017433513421565294,
    'PiStacking_T': 0.027523474767804146,
    'PiCation_lring': 0.009127387776970863,
    'PiCation_pring': 0.001319622853770852,
    'HBond_ldon': 0.0827205702662468,
    'HBond_pdon': 0.12345150858163834,
    'SaltBridge_lneg': 0.12117615342140198,
    'SaltBridge_pneg': 0.06078973412513733,
    'XBond': 0.0020935165230184793
}

In [5]:
module = ModelingModule('model_weights/model.tar', 'cuda',
                        score_threshold=score_threshold)

In [6]:
failed = []
for rec_id, lig_id in tqdm(split_file.items()):
    try:
        rec_path = os.path.join('dataset/crossdock', rec_id + '_rec.pdb')
        lig_path = os.path.join('dataset/crossdocked_pocket10', lig_id)
        out_path = os.path.join('dataset/new_pharmacophores', rec_id + 'pt')
        
        p_model = module.run(rec_path, ref_ligand_path=lig_path)
        p_model.save(out_path)
    except:
        failed.append((rec_id, lig_id))

  0%|          | 46/15307 [01:34<7:34:51,  1.79s/it] 